## Validación con Prestaciones (V2): KPIs 2025 y Cohorte
### Pagado es booleano (TRUE/FALSE), no monto.
### Los montos abonados no se pueden calcular con este CSV; trabajamos con conteos por paciente y cohorte.

# Setup

In [9]:
# Paths & imports

from pathlib import Path
import pandas as pd
import numpy as np

# Siempre funciona estando dentro o fuera de /notebooks
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
PREST_PATH = ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv"

assert PREST_PATH.exists(), f"No encuentro: {PREST_PATH}"

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", None)


In [10]:
# Carga y columnas clave

dfp = pd.read_csv(PREST_PATH, low_memory=False)

# Fechas
fecha      = pd.to_datetime(dfp.get('Fecha'), errors='coerce', dayfirst=True)
fec_estado = pd.to_datetime(dfp.get('Fec_Estado'), errors='coerce', dayfirst=True)

# Año/mes para presupuesto (Fecha) y pago (Fec_Estado; si falta, cae a Fecha)
anio_ppto = fecha.dt.year
mes_ppto  = fecha.dt.month
anio_pago = fec_estado.dt.year.fillna(anio_ppto)
mes_pago  = fec_estado.dt.month.fillna(mes_ppto)

# Pagado booleano robusto
pag_str  = dfp.get('Pagado', pd.Series(index=dfp.index, dtype=object)).astype(str).str.strip().str.upper()
pag_bool = pag_str.map({'TRUE': True, 'FALSE': False, '1': True, '0': False, 'SI': True, 'NO': False}).fillna(False)

# RUT
rut_col = 'RUT' if 'RUT' in dfp.columns else ('RutBeneficiario' if 'RutBeneficiario' in dfp.columns else None)
assert rut_col is not None, "No encuentro columna de RUT."
print("Filas Prestaciones:", len(dfp), "| Cols:", len(dfp.columns), "| RUT:", rut_col)


Filas Prestaciones: 997309 | Cols: 31 | RUT: RUT


In [11]:
# QA rápido de Pagado/Estado

def top_vals(s, n=10):
    return (s.astype(str).str.strip().str.upper()
            .replace({'': '(VACÍO)', 'NAN': '(NULO)'})
            .value_counts().head(n))

print("TOP Pagado:\n", top_vals(dfp['Pagado']))
print("\nTOP Estado:\n", top_vals(dfp.get('Estado', pd.Series(dtype=object))))


TOP Pagado:
 Pagado
FALSE    504190
TRUE     493119
Name: count, dtype: int64

TOP Estado:
 Estado
DIAGNOSTICADA    507609
TERMINADA        487939
INICIADA            864
INDICADA            838
GARANTIZADA          58
SIN TERMINAR          1
Name: count, dtype: int64


# KPIs 2025

In [12]:
# KPIs por paciente y series

# Detectar una columna de monto de presupuesto (para exigir >0). Si no hay, basta con la existencia.
m_ppto = None
for cand in ['Valor_Total','Precio_Total','Valor_Prest','Precio']:
    if cand in dfp.columns:
        m_ppto = cand; break

if m_ppto is not None:
    monto = pd.to_numeric(dfp[m_ppto], errors='coerce')
    mask_ppto_2025 = anio_ppto.eq(2025) & (monto > 0)
else:
    mask_ppto_2025 = anio_ppto.eq(2025)

mask_pago_2025 = anio_pago.eq(2025) & pag_bool

# KPIs por paciente único
pacientes_ppto_2025 = dfp.loc[mask_ppto_2025, rut_col].dropna().astype(str).nunique()
pacientes_pago_2025 = dfp.loc[mask_pago_2025, rut_col].dropna().astype(str).nunique()

# Series mensuales (pacientes únicos por mes)
serie_ppto = (dfp.loc[mask_ppto_2025, [rut_col]]
                .assign(_mes=mes_ppto[mask_ppto_2025].astype('Int64'))
                .dropna().drop_duplicates()
                .groupby('_mes')[rut_col].nunique()
                .reindex(range(1,13), fill_value=0))

serie_pago = (dfp.loc[mask_pago_2025, [rut_col]]
                .assign(_mes=mes_pago[mask_pago_2025].astype('Int64'))
                .dropna().drop_duplicates()
                .groupby('_mes')[rut_col].nunique()
                .reindex(range(1,13), fill_value=0))

# Top convenios entre quienes pagaron en 2025
if 'Convenio' in dfp.columns:
    top_conv_pago = (dfp.loc[mask_pago_2025, [rut_col,'Convenio']]
                       .dropna()
                       .drop_duplicates(rut_col)['Convenio']
                       .astype(str).str.upper()
                       .value_counts().head(10))
else:
    top_conv_pago = pd.Series(dtype='int64')

kpis_2025 = {
    'pacientes_ppto_creado_2025': int(pacientes_ppto_2025),
    'pacientes_con_abono_2025'  : int(pacientes_pago_2025),
    'nota' : 'Pagos 2025 incluyen arrastre de presupuestos antiguos (no linkeamos por ID).'
}

print("KPIs 2025 (conteos por paciente):", kpis_2025)
print("\nPacientes con ppto por mes (2025):\n", serie_ppto)
print("\nPacientes con abono por mes (2025):\n", serie_pago)
print("\nTop convenios (pagaron en 2025):\n", top_conv_pago)


KPIs 2025 (conteos por paciente): {'pacientes_ppto_creado_2025': 4330, 'pacientes_con_abono_2025': 4912, 'nota': 'Pagos 2025 incluyen arrastre de presupuestos antiguos (no linkeamos por ID).'}

Pacientes con ppto por mes (2025):
 _mes
1     387
2     483
3     640
4     660
5     590
6     491
7     761
8     630
9     463
10    615
11    531
12    535
Name: RUT, dtype: int64

Pacientes con abono por mes (2025):
 _mes
1      549
2      715
3     1012
4      953
5      894
6      715
7     1139
8      881
9      740
10     922
11     838
12     814
Name: RUT, dtype: int64

Top convenios (pagaron en 2025):
 Convenio
FONASA                                3077
ISAPRE                                 650
CAJA COMPENSACION LOS ANDES            233
CAJA LOS HEROES                        114
TARJETA SOY PROVIDENCIA                 65
DUOC UC                                 50
PORTAL ORTODONCIA                       22
BANCO DE CHILE                          19
NOVOFARMA SERVICE S.A             

In [13]:
# Cohorte 2025
# Cohorte: pacientes que crearon Ppto en 2025 (mask_ppto_2025)
cohorte_2025      = set(dfp.loc[mask_ppto_2025, rut_col].dropna().astype(str).unique())
pagadores_2025    = set(dfp.loc[mask_pago_2025, rut_col].dropna().astype(str).unique())
pag_en_cohorte    = len(cohorte_2025 & pagadores_2025)
n_cohorte         = len(cohorte_2025)
tasa_cohorte_2025 = (pag_en_cohorte / n_cohorte) if n_cohorte else np.nan

print({
    "cohorte_pacientes_ppto_2025": n_cohorte,
    "pagadores_2025_en_cohorte"  : pag_en_cohorte,
    "tasa_pago_en_misma_cohorte" : None if pd.isna(tasa_cohorte_2025) else round(tasa_cohorte_2025, 3)
})


{'cohorte_pacientes_ppto_2025': 4330, 'pagadores_2025_en_cohorte': 3951, 'tasa_pago_en_misma_cohorte': 0.912}


# (Opcional) Enriquecimiento con Clientes

In [ ]:
# Join opcional (APAGADA por defecto)
RUN_COMPARATIVA = False  # ← pon True si quieres ejecutar

if RUN_COMPARATIVA:
    CLIENTES_PATH = ROOT / "data" / "raw" / "Tab_Clientes(2).csv"
    assert CLIENTES_PATH.exists(), f"No encuentro: {CLIENTES_PATH}"
    dfc = pd.read_csv(CLIENTES_PATH, low_memory=False)

    # Mapping simple por RUT → Empresa y Comuna/Región (primera no-nula por RUT)
    key_cli = 'RutBeneficiario' if 'RutBeneficiario' in dfc.columns else 'RUT'
    assert key_cli in dfc.columns, "No encuentro columna RUT en clientes."

    map_empresa = (dfc[[key_cli,'Empresa']]
                   .dropna(subset=[key_cli])
                   .drop_duplicates(subset=[key_cli])
                   .set_index(key_cli)['Empresa'])

    dfp['_RUT_str'] = dfp[rut_col].astype(str)
    dfp['Empresa_from_clientes'] = dfp['_RUT_str'].map(map_empresa)

    # Top Empresas entre quienes pagaron en 2025 (vía clientes)
    top_emp_pago = (dfp.loc[mask_pago_2025, ['_RUT_str','Empresa_from_clientes']]
                      .dropna()
                      .drop_duplicates('_RUT_str')['Empresa_from_clientes']
                      .astype(str).str.upper()
                      .value_counts().head(15))
    print("Top Empresas (pagaron 2025, via clientes):\n", top_emp_pago)


Top Empresas (pagaron 2025, via clientes):
 Empresa_from_clientes
FONASA                                2819
ISAPRE                                 607
CAJA COMPENSACION LOS ANDES            231
CAJA LOS HEROES                        118
DUOC UC                                 45
TARJETA SOY PROVIDENCIA                 36
BANCO DE CHILE                          19
NOVOFARMA SERVICE S.A                   15
PORTAL ORTODONCIA                       13
HOSPITAL DR. LUIS CALVO MACKENNA        12
CAJA 18 DE SEPTIEMBRE TRABAJADORES      11
INMOBILIARIA SYCAR S.A.                  9
CAJA 18 DE SEPTIEMBRE PENSIONADOS        6
TARJETA NACIONAL ESTUDIANTIL             6
COMPÁS COMUNICACIONES LIMITADA           6
Name: count, dtype: int64


# Export mini-resumen

In [14]:
OUT_DIR = ROOT / "data" / "interim"
OUT_DIR.mkdir(parents=True, exist_ok=True)

(serie_ppto.rename("pacientes_ppto_mes_2025")
           .to_csv(OUT_DIR / "prest_serie_ppto_2025.csv", index=True))
(serie_pago.rename("pacientes_pago_mes_2025")
           .to_csv(OUT_DIR / "prest_serie_pago_2025.csv", index=True))

pd.Series(kpis_2025).to_csv(OUT_DIR / "prest_kpis_2025_conteos.csv")
print("Exportados a:", OUT_DIR)


Exportados a: /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/data/interim


### 🗃️ Histórico V1 (no ejecutar)

In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    from pathlib import Path
    import pandas as pd
    import numpy as np

    ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
    fp = ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv"

    # 1) Cargar e inspeccionar
    dfp = pd.read_csv(fp, low_memory=False)
    print("Cols prestaciones:", list(dfp.columns)[:30])

    # 2) Mapea columnas (ajusta si difiere)
    MAP = {
        "rut":      ["RutBeneficiario","Rut","RUT","IdPaciente","PacienteID"],
        "fecha":    ["Fecha","FechaAtencion","FechaPpto","FechaPresupuesto","FechaRegistro","Fec_Atencion","Fec_Ppto"],
        "tipo":     ["Tipo","TipoPrestacion","Movimiento","Estado","Evento"],
        # Monto total del Ppto y Monto abonado (ajusta si existen)
        "monto_ppto":   ["MontoPresupuestado","TotalPresupuesto","MontoPpto","Total","Valor"],
        "monto_abono":  ["MontoAbonado","Abonado","Pagado","MontoPago","MontoAvance"],
    }

    def pick(d, candidates):
        for c in candidates:
            if c in d.columns: return c
        return None

    col_rut   = pick(dfp, MAP["rut"])
    col_fecha = pick(dfp, MAP["fecha"])
    col_tipo  = pick(dfp, MAP["tipo"])
    col_mp    = pick(dfp, MAP["monto_ppto"])
    col_ma    = pick(dfp, MAP["monto_abono"])

    print("Usando columnas ->", dict(rut=col_rut, fecha=col_fecha, tipo=col_tipo, monto_ppto=col_mp, monto_abono=col_ma))

    # 3) Parse fecha y cortar 2025
    f = pd.to_datetime(dfp[col_fecha], errors="coerce", dayfirst=True)
    dfp["_anio"] = f.dt.year
    dfp["_mes"]  = f.dt.month
    dfp_2025 = dfp[dfp["_anio"].eq(2025)].copy()

    # 4) Derivar KPIs aproximando las tarjetas de PBI
    #   - Define reglas para "ppto creado", "abono" y "avance"
    def flag_contains(s, patterns):
        s = s.astype(str).str.upper()
        pat = "|".join([p.upper() for p in patterns])
        return s.str.contains(pat, regex=True)

    if col_tipo is not None:
        is_ppto  = flag_contains(dfp_2025[col_tipo], ["PRESUPUESTO","PPTO","CREADO"])
        is_abono = flag_contains(dfp_2025[col_tipo], ["ABONO","PAGO","COBRO"])
        is_avance= flag_contains(dfp_2025[col_tipo], ["AVANCE"])
    else:
        # Si no existe 'tipo', nos vamos por montos disponibles
        is_ppto  = dfp_2025[col_mp].notna() if col_mp else pd.Series(False, index=dfp_2025.index)
        is_abono = dfp_2025[col_ma].notna() if col_ma else pd.Series(False, index=dfp_2025.index)
        is_avance= pd.Series(False, index=dfp_2025.index)

    # 5) KPIs (2025)
    kpis = {}

    # Pacientes con ppto creado en 2025
    if col_rut:
        kpis["pacientes_ppto_creado"] = int(dfp_2025.loc[is_ppto, col_rut].nunique())
        kpis["pacientes_con_abono"]   = int(dfp_2025.loc[is_abono, col_rut].nunique())
        kpis["pacientes_con_avance"]  = int(dfp_2025.loc[is_avance, col_rut].nunique())

    # Montos
    if col_mp:
        kpis["monto_total_ppto_2025"] = float(dfp_2025.loc[is_ppto, col_mp].sum())
    if col_ma:
        kpis["monto_total_abonos_2025"] = float(dfp_2025.loc[is_abono, col_ma].sum())

    # % Avance abonado (métricas de PBI suelen ser sum(abonos)/sum(pptos))
    if col_mp and col_ma and kpis.get("monto_total_ppto_2025", 0) > 0:
        kpis["pct_avance_abonado"] = kpis["monto_total_abonos_2025"] / kpis["monto_total_ppto_2025"]

    print("KPIs 2025 estimados:", kpis)

    # 6) Serie por mes (pacientes con ppto por mes)
    if col_rut:
        serie_mes = (dfp_2025.loc[is_ppto, [col_rut, "_mes"]]
                    .dropna()
                    .drop_duplicates()
                    .groupby("_mes")[col_rut].nunique()
                    .reindex(range(1,13), fill_value=0))
        print("Pacientes con ppto creado por mes (2025):")
        print(serie_mes)

    # 7) Top convenios por paciente en 2025
    #    Join con clientes para tomar Empresa/Convenio limpio
    dfc = pd.read_csv(ROOT / "data" / "raw" / "Tab_Clientes(2).csv", low_memory=False)
    # Tomar solo columnas necesarias
    join_cols = ["RutBeneficiario","Empresa"]
    join_cols = [c for c in join_cols if c in dfc.columns]
    dfc_j = dfc[join_cols].copy()

    if col_rut and join_cols:
        dfp_2025_u = dfp_2025.loc[is_ppto, [col_rut]].dropna().drop_duplicates()
        top_conv = (dfp_2025_u.merge(dfc_j, left_on=col_rut, right_on="RutBeneficiario", how="left")
                    .Empresa.astype(str).str.upper().value_counts().head(10))
        print("Top convenios (pacientes con ppto 2025):")
        print(top_conv)


Cols prestaciones: ['OT', 'RUT', 'Paciente', 'CodigoPrest', 'Pieza', 'Cara', 'Fecha', 'RutDentista', 'Dentista', 'Clinica', 'Precio', 'Precio_Lab', 'Precio_Total', 'Valor_Prest', 'Valor_Lab', 'Valor_Total', 'Pagado', 'Estado', 'Fec_Estado', 'Especialidad', 'Prestacion', 'TipoConvenio', 'Convenio', 'Promocion', 'LiquidadoTratante', 'LiquidadoDiag', 'LiquidadoLab', 'MargenLab', 'Margen', 'Edad']
Usando columnas -> {'rut': 'RUT', 'fecha': 'Fecha', 'tipo': 'Estado', 'monto_ppto': None, 'monto_abono': 'Pagado'}
KPIs 2025 estimados: {'pacientes_ppto_creado': 0, 'pacientes_con_abono': 0, 'pacientes_con_avance': 0, 'monto_total_abonos_2025': 0.0}
Pacientes con ppto creado por mes (2025):
_mes
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
Name: RUT, dtype: int64
Top convenios (pacientes con ppto 2025):
Series([], Name: count, dtype: int64)


In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    from pathlib import Path
    import pandas as pd
    import numpy as np

    ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
    dfp = pd.read_csv(ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv", low_memory=False)

    # Parseo mínimo
    dfp['Fecha'] = pd.to_datetime(dfp['Fecha'], errors='coerce', dayfirst=True)
    dfp['_anio'] = dfp['Fecha'].dt.year
    dfp['_mes']  = dfp['Fecha'].dt.month

    # Montos: ppto vs abonos
    m_ppto = 'Valor_Total' if 'Valor_Total' in dfp.columns else ('Precio_Total' if 'Precio_Total' in dfp.columns else None)
    m_abon = 'Pagado' if 'Pagado' in dfp.columns else None
    assert m_abon is not None, "No encuentro columna de abonos (Pagado)."

    # 2025
    d25 = dfp[dfp['_anio'].eq(2025)].copy()

    # Limpieza básica de montos
    for c in [m_ppto, m_abon]:
        if c: d25[c] = pd.to_numeric(d25[c], errors='coerce').fillna(0)

    # KPI por PACIENTE en 2025 (aprox tarjetas PBI)
    agg_pac = (d25
        .groupby('RUT')[[m_ppto, m_abon]]
        .sum(min_count=1)
        .fillna(0)
        .rename(columns={m_ppto:'ppto_2025', m_abon:'abon_2025'}))

    kpis = {
        'pacientes_ppto_creado'   : int((agg_pac['ppto_2025'] > 0).sum()),
        'pacientes_con_abono'     : int((agg_pac['abon_2025'] > 0).sum()),
        'pacientes_ppto_en_avance': int(((agg_pac['abon_2025'] > 0) & (agg_pac['abon_2025'] < agg_pac['ppto_2025'])).sum()),
        'monto_total_ppto_2025'   : float(agg_pac['ppto_2025'].sum()),
        'monto_total_abonos_2025' : float(agg_pac['abon_2025'].sum()),
    }
    kpis['pct_avance_abonado'] = (kpis['monto_total_abonos_2025'] / kpis['monto_total_ppto_2025']) if kpis['monto_total_ppto_2025']>0 else np.nan
    print("KPIs 2025 (aprox paciente):", kpis)

    # Serie mensual (pacientes con presupuesto por mes)
    serie_mes = (d25.groupby(['_mes','RUT'])[m_ppto].sum().reset_index()
                .query(f"{m_ppto} > 0")
                .groupby('_mes')['RUT'].nunique()
                .reindex(range(1,13), fill_value=0))
    print("\nPacientes con ppto por mes (2025):")
    print(serie_mes)

    # Top convenios por paciente (usando Prestaciones directamente)
    top_conv = (d25.groupby('RUT')['Convenio'].agg(lambda s: str(s.dropna().iloc[0]) if len(s.dropna()) else '(EN BLANCO)')
                .value_counts().head(10))
    print("\nTop convenios por paciente (2025, Prestaciones):")
    print(top_conv)

    # (Opcional) Comparar con 'Empresa' de Clientes:
    dfc = pd.read_csv(ROOT / "data" / "raw" / "Tab_Clientes(2).csv", low_memory=False)
    top_conv_emp = (d25[['RUT']].drop_duplicates()
                    .merge(dfc[['RutBeneficiario','Empresa']], left_on='RUT', right_on='RutBeneficiario', how='left')
                    .Empresa.astype(str).str.upper().value_counts().head(10))
    print("\nTop convenios por paciente (2025, via Empresa de Clientes):")
    print(top_conv_emp)


KPIs 2025 (aprox paciente): {'pacientes_ppto_creado': 4330, 'pacientes_con_abono': 3845, 'pacientes_ppto_en_avance': 3667, 'monto_total_ppto_2025': 1366253002.0, 'monto_total_abonos_2025': 14793.0, 'pct_avance_abonado': 1.0827423601884243e-05}

Pacientes con ppto por mes (2025):
_mes
1     387
2     483
3     640
4     660
5     590
6     491
7     761
8     630
9     463
10    615
11    531
12    535
Name: RUT, dtype: int64

Top convenios por paciente (2025, Prestaciones):
Convenio
FONASA                                2884
ISAPRE                                 595
(EN BLANCO)                            497
CAJA COMPENSACION LOS ANDES            208
CAJA LOS HEROES                        126
Tarjeta Soy Providencia                 68
DUOC UC                                 44
Portal Ortodoncia                       23
Banco de Chile                          18
CAJA 18 DE SEPTIEMBRE TRABAJADORES      16
Name: count, dtype: int64

Top convenios por paciente (2025, via Empresa de Client

In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    # Normalización de montos CLP con miles '.' y decimales ','
    import pandas as pd
    def parse_clp(s):
        if s is None: 
            return pd.Series(dtype='float64')
        return pd.to_numeric(
            s.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False),
            errors='coerce'
        )

    d25_norm = d25.copy()
    if m_abon:  d25_norm[m_abon] = parse_clp(d25_norm[m_abon])
    if m_ppto:  d25_norm[m_ppto] = parse_clp(d25_norm[m_ppto])

    agg_pac_n = (d25_norm
                .groupby('RUT')[[m_ppto, m_abon]]
                .sum(min_count=1).fillna(0)
                .rename(columns={m_ppto:'ppto_2025', m_abon:'abon_2025'}))

    kpis_n = {
        'pacientes_ppto_creado'   : int((agg_pac_n['ppto_2025'] > 0).sum()),
        'pacientes_con_abono'     : int((agg_pac_n['abon_2025'] > 0).sum()),
        'pacientes_ppto_en_avance': int(((agg_pac_n['abon_2025'] > 0) & (agg_pac_n['abon_2025'] < agg_pac_n['ppto_2025'])).sum()),
        'monto_total_ppto_2025'   : float(agg_pac_n['ppto_2025'].sum()),
        'monto_total_abonos_2025' : float(agg_pac_n['abon_2025'].sum()),
    }
    kpis_n['pct_avance_abonado'] = (kpis_n['monto_total_abonos_2025']/kpis_n['monto_total_ppto_2025']
                                    if kpis_n['monto_total_ppto_2025']>0 else None)
kpis_n


{'pacientes_ppto_creado': 4330,
 'pacientes_con_abono': 0,
 'pacientes_ppto_en_avance': 0,
 'monto_total_ppto_2025': 1366253002.0,
 'monto_total_abonos_2025': 0.0,
 'pct_avance_abonado': 0.0}

In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    # DIAGNÓSTICO: ¿Los abonos 2025 están fechados en Fec_Estado?
    import pandas as pd
    from pathlib import Path

    ROOT = Path.cwd().parent if Path.cwd().name=="notebooks" else Path.cwd()
    dfp = pd.read_csv(ROOT/"data"/"raw"/"ETL_vPrestaciones (2).csv", low_memory=False)

    def parse_clp_series(s):
        return pd.to_numeric(
            s.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False),
            errors='coerce'
        )

    fecha      = pd.to_datetime(dfp.get('Fecha'), errors='coerce', dayfirst=True)
    fec_estado = pd.to_datetime(dfp.get('Fec_Estado'), errors='coerce', dayfirst=True)
    pag        = parse_clp_series(dfp['Pagado']) if 'Pagado' in dfp.columns else pd.Series(dtype='float64')
    estado     = dfp.get('Estado', pd.Series(index=dfp.index, dtype=str)).astype(str).str.upper()

    print("Non-null Pagado:", int(pag.notna().sum()), " | Suma total (todas fechas):", float(pag.fillna(0).sum()))

    by_year_fecha = pag.groupby(fecha.dt.year).sum(min_count=1).dropna().sort_index()
    by_year_fest  = pag.groupby(fec_estado.dt.year).sum(min_count=1).dropna().sort_index()
    print("\nSuma Pagado por año (usando Fecha):\n", by_year_fecha.tail(6))
    print("\nSuma Pagado por año (usando Fec_Estado):\n", by_year_fest.tail(6))

    print("\nTop Estado con pago>0:\n", estado[pag>0].value_counts().head(10))

    mask_2025_fest = fec_estado.dt.year.eq(2025)
    print("\n2025 por Fec_Estado → filas con pago>0:", int((pag[mask_2025_fest]>0).sum()),
        " | suma:", float(pag[mask_2025_fest].sum()))


Non-null Pagado: 0  | Suma total (todas fechas): 0.0

Suma Pagado por año (usando Fecha):
 Series([], Name: Pagado, dtype: float64)

Suma Pagado por año (usando Fec_Estado):
 Series([], Name: Pagado, dtype: float64)

Top Estado con pago>0:
 Series([], Name: count, dtype: int64)

2025 por Fec_Estado → filas con pago>0: 0  | suma: 0.0


In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    from pathlib import Path
    import pandas as pd

    ROOT = Path.cwd().parent if Path.cwd().name=="notebooks" else Path.cwd()
    dfp = pd.read_csv(ROOT/"data"/"raw"/"ETL_vPrestaciones (2).csv", low_memory=False)

    def top_vals(s, n=20):
        return (s.astype(str).str.strip().str.upper()
                .replace({'': '(VACÍO)', 'NAN': '(NULO)'})
                .value_counts().head(n))

    print(">>> TOP valores 'Pagado':")
    print(top_vals(dfp['Pagado']))

    print("\n>>> TOP valores 'Estado':")
    print(top_vals(dfp['Estado']))

    print("\nMuestra cruda Pagado (primeras 10 celdas):")
    print(dfp['Pagado'].head(10).tolist())


>>> TOP valores 'Pagado':
Pagado
FALSE    504190
TRUE     493119
Name: count, dtype: int64

>>> TOP valores 'Estado':
Estado
DIAGNOSTICADA    507609
TERMINADA        487939
INICIADA            864
INDICADA            838
GARANTIZADA          58
SIN TERMINAR          1
Name: count, dtype: int64

Muestra cruda Pagado (primeras 10 celdas):
[True, True, True, True, True, True, True, True, True, True]


In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    # KPIs 2025 basados en Pagado (booleano) — conteos por paciente
    from pathlib import Path
    import pandas as pd
    import numpy as np

    ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
    dfp = pd.read_csv(ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv", low_memory=False)

    # --- Parse de fechas
    fecha      = pd.to_datetime(dfp.get('Fecha'), errors='coerce', dayfirst=True)
    fec_estado = pd.to_datetime(dfp.get('Fec_Estado'), errors='coerce', dayfirst=True)

    # Año/mes de "presupuesto/registro" y de "pago"
    anio_ppto = fecha.dt.year
    mes_ppto  = fecha.dt.month

    anio_pago = fec_estado.dt.year.fillna(anio_ppto)  # si no hay Fec_Estado, caer a Fecha
    mes_pago  = fec_estado.dt.month.fillna(mes_ppto)

    # --- Pagado como booleano robusto
    pag_bool = (dfp['Pagado']
                .astype(str).str.strip().str.upper()
                .map({'TRUE': True, 'FALSE': False})
                .fillna(False))

    # --- Montos de referencia del "presupuesto" (solo para identificar que hay registro)
    m_ppto = None
    for cand in ['Valor_Total', 'Precio_Total', 'Valor_Prest', 'Precio']:
        if cand in dfp.columns:
            m_ppto = cand
            break

    if m_ppto is None:
        # Si no hay montos, usamos la mera existencia de filas como "registro de ppto"
        tiene_ppto_2025 = anio_ppto.eq(2025)
    else:
        # Consideramos "ppto" si el monto es > 0
        monto = pd.to_numeric(dfp[m_ppto], errors='coerce')
        tiene_ppto_2025 = anio_ppto.eq(2025) & (monto > 0)

    # --- Flags 2025
    tiene_pago_2025 = anio_pago.eq(2025) & pag_bool

    # --- KPIs por PACIENTE
    rut_col = 'RUT' if 'RUT' in dfp.columns else 'RutBeneficiario'
    assert rut_col in dfp.columns, "No encuentro columna de RUT para identificar pacientes."

    pacientes_ppto_2025 = dfp.loc[tiene_ppto_2025, rut_col].dropna().astype(str).nunique()
    pacientes_pago_2025 = dfp.loc[tiene_pago_2025, rut_col].dropna().astype(str).nunique()

    # Serie mensual (n° pacientes únicos por mes)
    serie_ppto = (dfp.loc[tiene_ppto_2025, [rut_col]]
                    .assign(_mes=mes_ppto[tiene_ppto_2025].astype('Int64'))
                    .dropna()
                    .drop_duplicates()
                    .groupby('_mes')[rut_col].nunique()
                    .reindex(range(1,13), fill_value=0))

    serie_pago = (dfp.loc[tiene_pago_2025, [rut_col]]
                    .assign(_mes=mes_pago[tiene_pago_2025].astype('Int64'))
                    .dropna()
                    .drop_duplicates()
                    .groupby('_mes')[rut_col].nunique()
                    .reindex(range(1,13), fill_value=0))

    # Top convenios entre quienes pagaron en 2025
    if 'Convenio' in dfp.columns:
        top_conv_pago = (dfp.loc[tiene_pago_2025, ['RUT','Convenio']]
                        .dropna()
                        .drop_duplicates('RUT')['Convenio']
                        .astype(str).str.upper()
                        .value_counts().head(10))
    else:
        top_conv_pago = pd.Series(dtype='int64')

    kpis_bool = {
        'pacientes_ppto_creado_2025': int(pacientes_ppto_2025),
        'pacientes_con_abono_2025'  : int(pacientes_pago_2025),
        'tasa_pacientes_con_abono'  : (int(pacientes_pago_2025) / int(pacientes_ppto_2025)) if pacientes_ppto_2025 else np.nan,
    }

    print("KPIs 2025 (conteos por paciente, Pagado booleano):", kpis_bool)
    print("\nPacientes con ppto por mes (2025):\n", serie_ppto)
    print("\nPacientes con abono por mes (2025):\n", serie_pago)
    print("\nTop convenios (pacientes que pagaron en 2025):\n", top_conv_pago)


KPIs 2025 (conteos por paciente, Pagado booleano): {'pacientes_ppto_creado_2025': 4330, 'pacientes_con_abono_2025': 4912, 'tasa_pacientes_con_abono': 1.1344110854503464}

Pacientes con ppto por mes (2025):
 _mes
1     387
2     483
3     640
4     660
5     590
6     491
7     761
8     630
9     463
10    615
11    531
12    535
Name: RUT, dtype: int64

Pacientes con abono por mes (2025):
 _mes
1      549
2      715
3     1012
4      953
5      894
6      715
7     1139
8      881
9      740
10     922
11     838
12     814
Name: RUT, dtype: int64

Top convenios (pacientes que pagaron en 2025):
 Convenio
FONASA                                3077
ISAPRE                                 650
CAJA COMPENSACION LOS ANDES            233
CAJA LOS HEROES                        114
TARJETA SOY PROVIDENCIA                 65
DUOC UC                                 50
PORTAL ORTODONCIA                       22
BANCO DE CHILE                          19
NOVOFARMA SERVICE S.A                   14


In [16]:
# --- Chequeo Titular vs Beneficiario (opcional) ---
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
CLIENTES_PATH = ROOT / "data" / "raw" / "Tab_Clientes(2).csv"
if CLIENTES_PATH.exists():
    dfc = pd.read_csv(CLIENTES_PATH, low_memory=False)
    if {'RutTitular','RutBeneficiario'}.issubset(dfc.columns):
        a = dfc['RutTitular'].astype(str)
        b = dfc['RutBeneficiario'].astype(str)
        prop_iguales = (a == b).mean()
        print("Proporción titular == beneficiario:", round(prop_iguales, 3))

        dist = (dfc.groupby('RutTitular')['RutBeneficiario']
                  .nunique().describe(percentiles=[.5,.9,.95]))
        print("Beneficiarios distintos por titular (describe):\n", dist)
    else:
        print("No están ambas columnas en Clientes.")
else:
    print("No encuentro", CLIENTES_PATH)


Proporción titular == beneficiario: 0.86
Beneficiarios distintos por titular (describe):
 count    50744.000000
mean         1.143524
std          0.398490
min          1.000000
50%          1.000000
90%          2.000000
95%          2.000000
max          5.000000
Name: RutBeneficiario, dtype: float64


In [4]:
# === QA AUTOCONTENIDO: carga, parsing y pruebas de alineación ===
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
PREST_PATH = ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv"
assert PREST_PATH.exists(), f"No encuentro: {PREST_PATH}"

# --- Carga base de Prestaciones
dfp = pd.read_csv(PREST_PATH, low_memory=False)

# Columnas clave (robustas a mayúsculas/minúsculas)
cols = {c.lower(): c for c in dfp.columns}
rut_col     = cols.get('rut', 'RUT' if 'RUT' in dfp.columns else None)
fecha_col   = cols.get('fecha', 'Fecha' if 'Fecha' in dfp.columns else None)
fecest_col  = cols.get('fec_estado', 'Fec_Estado' if 'Fec_Estado' in dfp.columns else None)
estado_col  = cols.get('estado', 'Estado' if 'Estado' in dfp.columns else None)
pagado_col  = cols.get('pagado', 'Pagado' if 'Pagado' in dfp.columns else None)
ot_col      = cols.get('ot', 'OT' if 'OT' in dfp.columns else None)

needed = [rut_col, fecha_col, estado_col, pagado_col]
assert all(x in dfp.columns for x in needed), f"Faltan columnas mínimas: {needed}"

# --- Parseo de fechas
dfp['_fecha']     = pd.to_datetime(dfp[fecha_col], errors='coerce', dayfirst=True)
dfp['_fecestado'] = pd.to_datetime(dfp.get(fecest_col), errors='coerce', dayfirst=True) if fecest_col in dfp.columns else pd.NaT

# --- Años derivados
dfp['_anio_fecha']     = dfp['_fecha'].dt.year
dfp['_anio_fecestado'] = dfp['_fecestado'].dt.year

# --- Booleano de pago desde 'Pagado' (TRUE/FALSE u otras variantes)
def to_bool(x):
    if pd.isna(x): return False
    s = str(x).strip().upper()
    if s in {'TRUE','1','SI','SÍ','YES'}: return True
    if s in {'FALSE','0','NO'}: return False
    return False

dfp['_pag_bool'] = dfp[pagado_col].map(to_bool)

# --- QA-1: “ppto 2025” solo en ciertos ESTADOS (DIAGNOSTICADA/INICIADA)
estados_ppto = {"DIAGNOSTICADA", "INICIADA"}
mask_ppto_2025 = (dfp['_anio_fecha'] == 2025) & (dfp[estado_col].astype(str).str.upper().isin(estados_ppto))
p_pp_to = dfp.loc[mask_ppto_2025, rut_col].dropna().astype(str).nunique()
print("Pacientes ppto 2025 (solo estados DIAGNOSTICADA/INICIADA):", p_pp_to)

# --- QA-2: ¿y si cuentan OT (órdenes) en vez de paciente?
if ot_col in dfp.columns:
    ot_ppto_2025 = dfp.loc[dfp['_anio_fecha'] == 2025, ot_col].dropna().astype(str).nunique()
    # pagos en 2025 por Fec_Estado + Pagado True
    mask_pago_2025 = (dfp['_anio_fecestado'] == 2025) & (dfp['_pag_bool'])
    ot_pago_2025 = dfp.loc[mask_pago_2025, ot_col].dropna().astype(str).nunique()
    print("OT con ppto 2025:", ot_ppto_2025, " | OT con pago 2025:", ot_pago_2025)
else:
    print("No hay columna OT en Prestaciones; QA-2 no aplica.")

# --- (Extra) Tus métricas “por paciente” para comparar rápidamente:
pac_ppto_2025 = dfp.loc[dfp['_anio_fecha'] == 2025, rut_col].dropna().astype(str).nunique()
pac_pago_2025 = dfp.loc[(dfp['_anio_fecestado'] == 2025) & dfp['_pag_bool'], rut_col].dropna().astype(str).nunique()
print("Paciente único: ppto_2025 =", pac_ppto_2025, " | pago_2025 =", pac_pago_2025)



Pacientes ppto 2025 (solo estados DIAGNOSTICADA/INICIADA): 3242
OT con ppto 2025: 6090  | OT con pago 2025: 5950
Paciente único: ppto_2025 = 4564  | pago_2025 = 4862


In [5]:
# === Export NB-06: Cohortes y QA de reglas ===

import pandas as pd
from pathlib import Path

# Definir ruta a carpeta reports
reports_path = Path(ROOT, "reports")
reports_path.mkdir(parents=True, exist_ok=True)

# Construir DataFrame con resultados globales de cohortes
df_prest_val = pd.DataFrame({
    "Corte": ["2025"],
    "Pacientes_con_Ppto": [4564],
    "Pacientes_con_Pago": [4862],
    "Pacientes_con_Ambos": [3951],
    "Tasa_Conversion": [3951 / 4564]  # ~0.91
})

# Exportar a CSV
export_path = reports_path / "nb06_prestaciones_validacion.csv"
df_prest_val.to_csv(export_path, index=False)

print("Archivo exportado en:", export_path)
df_prest_val


Archivo exportado en: /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/reports/nb06_prestaciones_validacion.csv


,Corte,Pacientes_con_Ppto,Pacientes_con_Pago,Pacientes_con_Ambos,Tasa_Conversion
0,2025,4564,4862,3951,0.865688
